In [1]:
import h5py
import pandas as pd
import shutil
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

2023-08-16 10:38:23.319294: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-16 10:38:23.991501: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-08-16 10:38:23.991570: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-08-16 10:38:23.991575: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(10*1000)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
TRAIN_OLD = f'/data/HSC/HSC_v6/step2A/127x127/5x127x127_training_with_mags_normalized.hdf5'
VAL_OLD = f'/data/HSC/HSC_v6/step2A/127x127/5x127x127_validation_with_mags_normalized.hdf5'
TEST_OLD = f'/data/HSC/HSC_v6/step2A/127x127/5x127x127_testing_with_mags_normalized.hdf5'
TRAIN_NEW = f'/data/HSC/HSC_v6/step2A/127x127/5x127x127_training_with_mags_normalized_by_band.hdf5'
VAL_NEW = f'/data/HSC/HSC_v6/step2A/127x127/5x127x127_validation_with_mags_normalized_by_band.hdf5'
TEST_NEW = f'/data/HSC/HSC_v6/step2A/127x127/5x127x127_testing_with_mags_normalized_by_band.hdf5'
with h5py.File(TRAIN_OLD, 'r') as f:
    shutil.copyfile(TRAIN_OLD, TRAIN_NEW)
with h5py.File(VAL_OLD, 'r') as f:
    shutil.copyfile(VAL_OLD, VAL_NEW)
with h5py.File(TEST_OLD, 'r') as f:
    shutil.copyfile(TEST_OLD, TEST_NEW)

In [ ]:
with h5py.File(TRAIN_OLD, 'r') as f:
    train_len = len(f['object_id'])
with h5py.File(VAL_OLD, 'r') as f:
    val_len = len(f['object_id'])
with h5py.File(TEST_OLD, 'r') as f:
    test_len = len(f['object_id'])

In [ ]:
train_len

In [ ]:
scaler_list = [StandardScaler() for i in range(5)]

In [ ]:
feat_list = ['g_cmodel_mag', 'r_cmodel_mag', 'i_cmodel_mag', 'z_cmodel_mag', 'y_cmodel_mag']

In [ ]:
with h5py.File(TRAIN_NEW, 'a') as f:
    for i, feat in enumerate(feat_list):
        f[feat+'_normalized'] = scaler_list[i].fit_transform(f[feat][:].reshape(-1,1)).reshape(train_len)

In [ ]:
with h5py.File(VAL_NEW, 'a') as f:
    for i, feat in enumerate(feat_list):
        f[feat+'_normalized'] = scaler_list[i].transform(f[feat][:].reshape(-1,1)).reshape(val_len)

In [ ]:
with h5py.File(TEST_NEW, 'a') as f:
    for i, feat in enumerate(feat_list):
        f[feat+'_normalized'] = scaler_list[i].transform(f[feat][:].reshape(-1,1)).reshape(test_len)

In [ ]:
with h5py.File(TEST_NEW, 'r') as f:
    print(f['g_cmodel_mag_normalized'][:])